In [1]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np

In [2]:
class Style300WDataset(Dataset):
    def __init__(self, root_dir, variant='300W-Original', transform=None):
        """
        Args:
            root_dir (str): Path to 300W-Style dataset (the folder containing all style folders).
            variant (str): One of ['300W-Original', '300W-Gray', '300W-Light', '300W-Sketch'].
            transform (callable): Transform to apply to images.
        """
        self.image_dir = os.path.join(root_dir, variant)
        self.anno_dir = os.path.join(root_dir, '300W-Annotations')
        self.transform = transform
        self.samples = []

        for fname in os.listdir(self.image_dir):
            if fname.endswith('.jpg') or fname.endswith('.png'):
                image_path = os.path.join(self.image_dir, fname)
                pts_name = os.path.splitext(fname)[0] + '.pts'
                anno_path = os.path.join(self.anno_dir, pts_name)
                if os.path.exists(anno_path):
                    self.samples.append((image_path, anno_path))

    def __len__(self):
        return len(self.samples)

    def _read_pts(self, path):
        with open(path, 'r') as f:
            lines = f.readlines()
            coords = []
            reading = False
            for line in lines:
                if line.strip() == '{':
                    reading = True
                    continue
                if line.strip() == '}':
                    break
                if reading:
                    x, y = map(float, line.strip().split())
                    coords.append([x, y])
        return np.array(coords, dtype=np.float32)

    def __getitem__(self, idx):
        img_path, pts_path = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        landmarks = self._read_pts(pts_path)
        return image, torch.tensor(landmarks)


In [3]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = Style300WDataset(
    root_dir='/media/miju_chowdhury/Miju/Datasets/300W-Style/300W-Convert',   # <- change this to your actual path
    variant='300W-Sketch',
    transform=transform
)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Loop through a batch
for images, landmarks in loader:
    print("Image batch shape:", images.shape)       # e.g. [32, 3, 224, 224]
    print("Landmarks shape:", landmarks.shape)      # e.g. [32, 68, 2]
    break


ValueError: num_samples should be a positive integer value, but got num_samples=0